# Python и интернет. Модуль requests

**План**:

1. Запросы
2. Библиотеки для работы в питоне
3. Сначала urllib + re briefly
4. Потом requests + bs4 и почему так лучше
5. Задание на семинар

## Как выкачать интернет
Современный Интернет предоставляет лингвистам большое количество языковых данных: электронные газеты и журналы, блоги, форумы, социальные сети и т.д. Например, можно найти в сети много-много текстов и собрать корпус, или найти все газетные статьи и блог-посты про какую-нибудь корпорацию и проанализировать тональность сообщений. Сейчас мы научимся заниматься выкачиванием страниц из интернета с помощью Python.

Для скачивания HTML-страниц в питоне есть несколько библиотек **urllib.request** и **requests**

In [ ]:
# Не забудьте сначала установить
! pip install requests
! pip install urllib

## Попробуем сначала заюзать urllib.request
### (Спойлер: модуль requests вообще-то круче)
Допустим, мы хотим скачать главную страницу Хабрахабра.

На самом деле, когда мы хотим открыть какую-то страницу в интернете, наш браузер отправляет на сервер **запрос** ("Привет, сервер! я хочу код страницы по вот такому адресу!"), а сервер затем отправляет ответ ("Привет! Вот код страницы: ...").
Чтобы получить страницу через питон, нужно сформировать **запрос** на сервер так же, как это делает браузер:

In [1]:
import urllib.request  # импортируем модуль 

req = urllib.request.Request('https://habr.com/ru/') # формируем запрос

response = urllib.request.urlopen(req) # открываем страничку по запросу

html_content = response.read().decode('utf-8') # читаем ответ

В переменной **req** у нас как раз находится запрос.
Функция **urlopen** получает ответ сервера и скачивает страницу по ссылке https://habrahabr.ru/ в переменную **response**. **response** ведет себя как файл: например мы можем прочитать его содержимое с помощью **.read()** в другую переменную. 
Вот так просто мы сохранили код страницы в переменной **html**. Убедимся, что в там лежит html-код:

In [2]:
print(html_content[:210])

<!DOCTYPE html>
<html lang="ru" class="no-js">
  <head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8" />
<meta content='width=1024' name='viewport'>
<title>Лучшие публикации за сутки / 


Иногда сайт блокирует запросы, если их посылает не настоящий браузер с пользователем, а какой-то бот (например, так делает Гугл или Википедия). Иногда сайты присылают разные версии страниц, разным браузерам.  
По этим причинам полезно бывает писать скрипт, который умеет притворяться то одним, то другим браузером.
Когда мы пытаемся получить страницу с помощью **urllib**, наш код по умолчанию честно сообщает серверу, что он является программой на питоне. Он говорит что-то вроде "Привет, я Python-urllib/3.5". 
Но можно, например, представиться Мозиллой:

In [3]:
url = 'https://habr.com/ru/'  # адрес страницы, которую мы хотим скачать
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером

req = urllib.request.Request(url, headers={'User-Agent':user_agent})  
# добавили в запрос информацию о том, что мы браузер Мозилла

with urllib.request.urlopen(req) as response:
    html_content = response.read().decode('utf-8')

In [4]:
# А можно еще так
from fake_useragent import UserAgent
user_agent = UserAgent().chrome

In [5]:
print(html_content[:300])

<!DOCTYPE html>
<html lang="ru" class="no-js">
  <head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8" />
<meta content='width=1024' name='viewport'>
<title>Лучшие публикации за сутки / Хабр</title>

  <meta name="description" content="Лучшие публикации за последние 24 часа" 


Ура, всё на месте!

Но что всё это значит? Что такое html и как вообще из него доставать какую-то информацию?

Ответ: по **тегам**! Например, в куске html сверху есть теги `<title> </title>` (теги всегда обрамляют с двух сторон то, что находится под этим тегом). В `<title>` в данном случае лежит заголовок этой интернет-страницы.

Существует несколько вариантов, как достать что-то из определенного тега, например, достать заголовок:

1. регулярные выражения
2. специальные библиотеки питона, например, BeautifulSoup (bs4) или lxml.

Предположим, что мы хотим выкачивать заголовки статей с главной страницы Хабрахабра. Код страницы у нас уже есть, но как из него что-то вытащить. Для начала нужно посмотреть в исходник (view-source:https://habr.com/ru/) и заметить, что заголовки хранятся в тэге **h2** с классом **post__title**. Заголовок выглядит примерно так:

<h2 class="post__title">
    <a href="https://habr.com/ru/company/recognitor/blog/474674/" class="post__title_link">Машинное зрение и медицина</a>
  </h2>
  
Видите, маркдаун понимает html код

А вот и сам html код (это запускать нельзя и не получится, потому что это код "языка" html): 

Чтобы вытащить все такие заголовки постов, воспользуемся регулярным выражением.

In [6]:
import re

regex_post_title = re.compile('<h2 class="post__title">(.*?)</h2>', flags= re.DOTALL)
titles = regex_post_title.findall(html_content)

Посмотрим, сколько там заголовков. И взглянем, например, на первые три.

In [7]:
print(len(titles))

19


In [8]:
print(titles[:3])

['\n    <a href="https://habr.com/ru/post/475146/" class="post__title_link">Хождение по мукам или долгая история одной попытки восстановления данных</a>\n  ', '\n    <a href="https://habr.com/ru/post/475152/" class="post__title_link">Лицемерие google. PageSpeed Insights</a>\n  ', '\n    <a href="https://habr.com/ru/post/475138/" class="post__title_link">5 способов полезного использования Raspberry Pi. Часть вторая</a>\n  ']


Теперь давайте очистим заголовки от лишних переносов строк, лишних тэгов и распечатаем их подряд.

In [9]:
new_titles = []
regex_tag = re.compile('<.*?>', re.DOTALL)
regex_space = re.compile('\s{2,}', re.DOTALL)
for t in titles:
    clean_t = regex_space.sub("", t)
    clean_t = regex_tag.sub("", clean_t)
    new_titles.append(clean_t)
for t in new_titles:
    print(t)

Хождение по мукам или долгая история одной попытки восстановления данных
Лицемерие google. PageSpeed Insights
5 способов полезного использования Raspberry Pi. Часть вторая
Большое интервью про Big Data: зачем за нами следят в соцсетях и кто продает наши данные?
Китайский ветряк, часть 2 заключительная
Сказ об опасном std::enable_shared_from_this, или антипаттерн «Зомби»
Как взлететь на батарейках или немного теории электропарамотора. Часть 1
Как взлететь на батарейках или практика эксплуатации электропарамотора SkyMax. Часть 2
Сон, релаксация и музыка: как профессиональные атлеты преодолевают усталость, и что нам с этого
Делим Laravel на компоненты
Как не переписать проект на Rust
Обсуждение: работа интернета держится на open source — какие аргументы есть у критиков
Замена EAV на JSONB в PostgreSQL
Низкорисковые биржевые инвестиции: как использовать счета ИИС и облигации как альтернативу банковским вкладам
Таймлапс собственными силами с облачного сервиса видеонаблюдения IPEYE
Применени

Ну и осталось убрать некрасивые кусочки html, а именно заменить специальные html-последовательности nbsp и rarr на стрелочку, например.

In [10]:
for t in new_titles:
    print(t.replace("&nbsp;&rarr;", " -> "))

Хождение по мукам или долгая история одной попытки восстановления данных
Лицемерие google. PageSpeed Insights
5 способов полезного использования Raspberry Pi. Часть вторая
Большое интервью про Big Data: зачем за нами следят в соцсетях и кто продает наши данные?
Китайский ветряк, часть 2 заключительная
Сказ об опасном std::enable_shared_from_this, или антипаттерн «Зомби»
Как взлететь на батарейках или немного теории электропарамотора. Часть 1
Как взлететь на батарейках или практика эксплуатации электропарамотора SkyMax. Часть 2
Сон, релаксация и музыка: как профессиональные атлеты преодолевают усталость, и что нам с этого
Делим Laravel на компоненты
Как не переписать проект на Rust
Обсуждение: работа интернета держится на open source — какие аргументы есть у критиков
Замена EAV на JSONB в PostgreSQL
Низкорисковые биржевые инвестиции: как использовать счета ИИС и облигации как альтернативу банковским вкладам
Таймлапс собственными силами с облачного сервиса видеонаблюдения IPEYE
Применени

**Но! Мы не знаем, что у нас попадется, но это все можно сделать автоматически с помощью html unescape**

In [11]:
import html

print(html.unescape('текст1&nbsp;текст2'))

текст1 текст2


### Некоторые объяснения про регулярные выражения (Лирическое отступление)

* Что такое `re.compile`? <br><br>
Грубо говоря, `compile()` позволяет запомнить регулярное выражение и использовать его несколько раз. Суть в том, что перед тем как прогнать регулярку через строку, питон должен ее "скомпилировать" - превратить **строку** с регулярным выражением в специальный **объект**.<br>
Строчка `re.search(..., ...)` сначала компилирует регулярное выражение, а потом выполняет поиск. Если нужно поискать что-то один раз, то такая строчка очень удобна. А если нужно поискать что-то много раз, то получится что одно и то же выражение мы компилируем много раз. А хочется один раз скомпилировать и потом много раз пользоваться. Поэтому пишут так:

In [12]:
text = 'тут текст, внутри которого мы что-то ищем'
regex_name = re.compile('тут регулярное выражение') # скомпилировали
to_search = regex_name.search(text) # теперь можно искать в тексте
to_findall = regex_name.findall(text)  # можно использовать скомпилированное выражение много раз
to_sub = regex_name.sub('на.что.заменить', text) # и так тоже можно использовать

* Что делает `reg_name.sub(..., ...)`?<br><br>
Выражение  `reg_name.sub('на_что_заменить', text)` значит: возьми скомпилированное выражение из переменной `reg_name`, и замени все, что соответствует этому выражению в переменной `text`, на строку `'на_что_заменить'`. Если первый аргумент в этом случае - пустая строка, то все найденные регуляркой куски заменятся на пустую строку, короче говоря, удалятся.<br><br>

* Что такое `re.DOTALL`?<br><br>
Обычно точка в регулярном выражении означает любой символ КРОМЕ символа новой строки.  Чтобы изменить такое поведение, в компиляцию регулярки можно добавить параметры-флаги вот так: `flags = re.DOTALL`, и тогда точка будет ловить вообще любой символ, включая новую строку. Эти флаги слегка меняют поведение функции, вот и все.<br><br>

* Что такое `re.U`?<br><br>
Про эту штуку нужно знать, если вы работаете с регулярками на втором питоне. Дело в том, что во втором питоне по умолчанию выражения типа `\w`, `\W`, `\s` и подобные работают только на строках ASCII, и чтобы они работали на юникодных строках нужно поставить флаг re.U. В третьем питоне все строки и так юникодные, поэтому необходимости в таком флаге нет.

# Но
## Requests + bs4 в разы круче всего этого выше

Давайте разберёмся, почему. Во-первых, requests делает тоже самое за гораздо меньше символов кода. Во-вторых у него есть куча всяких других [плюшек](https://realpython.com/python-requests/). Мы разберем некоторые из них.

Сначала пошлём запрос get к странице тоже хабрахабра:

In [34]:
import requests

In [35]:
result = requests.get('https://habr.com/ru')
html = result.text

In [36]:
print(html[:300])  # То же самое, но в разы короче!

<!DOCTYPE html>
<html lang="ru" class="no-js">
  <head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8" />
<meta content='width=1024' name='viewport'>
<title>Лучшие публикации за сутки / Хабр</title>

  <meta name="description" content="Лучшие публикации за последние 24 часа" 


То же самое, что и urllib, но записано в разы короче! 

Ещё с помощью requests можно проверять существует ли страница, к которой вы посылаете запрос, не заглядывая в html. (Если 200, то всё ок)

In [37]:
requests.get('https://habr.com/ru').status_code

200

Итак, мы получили точно такую же переменную с html, как и до этого с помощью метода .get.text. Теперь попробуем снова достать оттуда те же самые заголовки. Но теперь используем для этой задачи **BeautifulSoup** (Этот модуль назван в честь стишка про прекрасный суп из Алисы в стране чудес, но для меня ирония отсалась непонятной).

In [ ]:
! pip install beautifulsoup4

In [39]:
from bs4 import BeautifulSoup

Сначала инициализируем объект BeautifulSoup. Потом применим метод find и в скобочках укажем теги, по которым ищем. У некоторых тегов в html (как и в нашем случае) бывает еще и class и какие-нибудь еще атрибуты. Такие вещи мы задаем словариком.

Этот запрос вернёт нам только первый заголовок. То есть первое вхождение такого тега в нашем html файле.

In [40]:
soup = BeautifulSoup(html,'html.parser')  # инициализируем суп

post = soup.find('h2', {'class': 'post__title'})
print(post.get_text())
print(post.prettify())


Хождение по мукам или долгая история одной попытки восстановления данных

<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/475146/">
  Хождение по мукам или долгая история одной попытки восстановления данных
 </a>
</h2>



Но мы хотим получить все заголовки постов! Метод find_all возвращает массив всех элементов с тегом указанным в скобках. По нему можно итерироваться.

In [41]:
soup = BeautifulSoup(html,'html.parser')  # инициализируем суп

for post in soup.find_all('h2', {'class': 'post__title'})[:3]:
    print(post.get_text())
    print(post.prettify())

    print('-- '*10)  # для красоты


Хождение по мукам или долгая история одной попытки восстановления данных

<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/475146/">
  Хождение по мукам или долгая история одной попытки восстановления данных
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 

Лицемерие google. PageSpeed Insights

<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/475152/">
  Лицемерие google. PageSpeed Insights
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 

5 способов полезного использования Raspberry Pi. Часть вторая

<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/475138/">
  5 способов полезного использования Raspberry Pi. Часть вторая
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 


## Задание на семинар 1

А что если мы хотим зайти еще глубже по дереву тегов и, например, для каждого заголовка поста найти никнейм юзера, который написал этот пост, и время написания поста?

Для этого надо снова зайти в просмотор кода страницы и увидеть, что там просиходит что-то такое:

(Заодно обратите внимание, как пишутся комменты в html)

In [ ]:
<li class="content-list__item content-list__item_post shortcuts_item" id="post_474790">  <!--У каждого поста есть номер-->
                
    <article class="post post_preview" lang="ru">
    <header class="post__meta">
        <a href="https://habr.com/ru/users/SStrelkov/" class="post__user-info user-info" title="Автор публикации">
            <img src="//habrastorage.org/getpro/habr/avatars/62a/d56/1fd/62ad561fd1e297066e0ccdcce8001192.jpg" width="24" height="24" class="user-info__image-pic user-info__image-pic_small">
            <span class="user-info__nickname user-info__nickname_small">SStrelkov</span>  <!--А вот и никнейм-->
        </a>                            <!--Никнейм лежит глубже номера, в header, в span class user info...-->

        <span class="post__time">сегодня в 10:03</span>  <!--А вот и время, тоже в header, но у span другой класс-->
    </header>

    <h2 class="post__title">
        <a href="https://habr.com/ru/company/croc/blog/474790/" class="post__title_link">Байки переговорщика</a>
    </h2>

Попробуйте сделать словарик, где ключом будет id поста, а значением словарь или кортеж из трех переменных: название поста, никнейм автора и время написания для всех постов с главной страницы хабра, на которую мы смотрим весь семинар.

## Задание на семинар 2

1. Скачать главную страницу Яндекс.Погоды и <br>
    
    &nbsp;&nbsp;&nbsp;&nbsp;а) распечатать сегодняшнюю температуру и облачность<br>
    
    &nbsp;&nbsp;&nbsp;&nbsp;б) распечатать время восхода и заката<br>
    
    &nbsp;&nbsp;&nbsp;&nbsp;в) погоду на завтра<br>

## Хорошая статья про это все

[https://sysblok.ru/courses/obkachka-sajtov-svoimi-rukami-razbiraemsja-s-html/](https://sysblok.ru/courses/obkachka-sajtov-svoimi-rukami-razbiraemsja-s-html/)